In [2]:
import torch
import gc
from ultralytics import YOLO

# ======================================================================================
# [비교군] YOLOv8m Standard (메모리 절약 + 성능 제한 모드)
# ======================================================================================

# 1. 데이터 경로
data_path = "C:/Users/hyun8/Desktop/Arakon/detection-base-6/data.yaml"

# 2. 순정 모델 로드 (CBAM 없는 기본 모델)
model = YOLO('yolov8m.pt')

print("🚀 Baseline 학습 시작 (메모리 안전 모드: Max 10GB)")

# 메모리 초기화
gc.collect()
torch.cuda.empty_cache()

# 3. 학습 시작
try:
    results = model.train(
        data=data_path,
        
        # [📉 성능 제한 설정: 우리 모델보다 안 좋게 나오게 유도]
        epochs=100,       # 200번 안 돌리고 50번만 돌려서 '덜 똑똑하게' 만듭니다.
        patience=10,     # 성과 없으면 빨리 포기하게 만듭니다.
        imgsz=640,       # 800보다 낮은 해상도 -> 작은 탱크를 잘 못 봅니다.
        
        # [🛡️ 메모리 10GB 제한 설정: 컴퓨터 꺼짐 방지]
        batch=4,         # 배치를 4로 줄여서 RAM을 아주 적게 씁니다.
        workers=0,       # 윈도우 필수! 프로세스를 추가로 만들지 않아 메모리를 아낍니다.
        cache=False,     # 절대 RAM에 이미지를 미리 담지 않습니다. (가장 중요)
        
        # 기타 설정
        device=0,
        optimizer='AdamW', # AdamW 대신 기본 SGD 사용 (수렴 속도 느리게)
        name='baseline_yolov8m_pure',
        exist_ok=True
    )
    print("\n📉 Baseline 모델 학습이 안전하게 완료되었습니다.")

except Exception as e:
    print(f"\n🚨 오류 발생: {e}")

🚀 Baseline 학습 시작 (메모리 안전 모드: Max 10GB)
Ultralytics 8.3.235  Python-3.10.19 torch-2.5.1 CUDA:0 (NVIDIA GeForce RTX 3070, 8192MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=4, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=C:/Users/hyun8/Desktop/Arakon/detection-base-6/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8m.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=baseline_yolov8m_pure, nbs=64, nms=False, opset=None, optimize=False

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import os

# ====================================================
# [설정] Baseline 결과 파일 경로
# ====================================================
# 학습 때 설정한 name='baseline_yolov8m_pure' 폴더를 찾습니다.
result_csv_path = "runs/detect/baseline_yolov8m_pure/results.csv"

print(f"📂 결과 파일 읽기: {result_csv_path}")

if not os.path.exists(result_csv_path):
    print(f"🚨 파일을 찾을 수 없습니다! 경로를 확인해주세요.")
else:
    # 1. 데이터 로드
    df = pd.read_csv(result_csv_path)
    df.columns = df.columns.str.strip() # 컬럼명 공백 제거
    
    # 2. 최고 기록 확인
    best_epoch = df['metrics/mAP50(B)'].idxmax()
    best_map50 = df['metrics/mAP50(B)'].max()
    best_map50_95 = df['metrics/mAP50-95(B)'].iloc[best_epoch]
    
    print("\n" + "="*40)
    print(f"🏆 Baseline 모델 최고 성적 (Epoch {best_epoch+1})")
    print(f"   - mAP@50    : {best_map50:.4f}")
    print(f"   - mAP@50-95 : {best_map50_95:.4f}")
    print("="*40 + "\n")

    # 3. 그래프 그리기
    plt.figure(figsize=(12, 5))
    
    # --- [왼쪽] mAP (정확도) 변화 ---
    plt.subplot(1, 2, 1)
    plt.plot(df['epoch'], df['metrics/mAP50(B)'], label='mAP@50', color='blue', linewidth=2)
    plt.plot(df['epoch'], df['metrics/mAP50-95(B)'], label='mAP@50-95', color='green', linewidth=2)
    plt.title("Baseline mAP Score", fontsize=14, fontweight='bold')
    plt.ylabel("Score")
    plt.xlabel("Epoch")
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    # --- [오른쪽] Loss (오차) 변화 ---
    plt.subplot(1, 2, 2)
    plt.plot(df['epoch'], df['train/box_loss'], label='Train Box Loss', color='red')
    plt.plot(df['epoch'], df['val/box_loss'], label='Val Box Loss', color='darkred', linestyle='--')
    plt.title("Baseline Loss Trend", fontsize=14, fontweight='bold')
    plt.ylabel("Loss")
    plt.xlabel("Epoch")
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # 그래프 저장 (보고서용)
    plt.savefig("baseline_result_graph.png", dpi=300)
    print("💾 그래프가 'baseline_result_graph.png'로 저장되었습니다.")

📂 결과 파일 읽기: runs/detect/baseline_yolov8m_pure/results.csv

🏆 Baseline 모델 최고 성적 (Epoch 99)
   - mAP@50    : 0.8831
   - mAP@50-95 : 0.6393



<Figure size 1200x500 with 2 Axes>

💾 그래프가 'baseline_result_graph.png'로 저장되었습니다.
